## Generating pandas data frame from vaa3d annotation.eswc file 

Also downsamples them to the corresponding dimensions

In [2]:
import os

import pandas as pd

import numpy as np

import tkinter as tk
import tkinter.filedialog as fdialog
from tkinter import simpledialog

In [3]:
anno_file=fdialog.askopenfile(initialdir='Z:\\', title='Select the eswc file containing the annotations').name 

xy = simpledialog.askfloat("Input", "What is the x and y resolution in um?",
                               minvalue=0.0, maxvalue=100)
z = simpledialog.askfloat("Input", "What is the z resolution in um?",
                               minvalue=0.0, maxvalue=100)
goal_xyz = simpledialog.askfloat("Input", "What do you want to downsample the resolution to '(in um)' ?",
                               minvalue=10, maxvalue=100)

outdir = fdialog.askdirectory(title='Please select the output directory')

In [4]:
anno=open(anno_file,'r')
anno_data=anno.readlines()
# heading is stored in anno_data[2], 1st line basically useless

headings=anno_data[2].split(' ')
annotations=[lines.split(' ') for lines in anno_data[3:]]

In [5]:
annotation_df=pd.DataFrame(annotations, columns=headings)

In [6]:
annotation_df.head()

,#n,type,x,y,z,radius,parent,seg_id,level,mode,timestamp,TFresindex
0,1388320,3,14013.111,2887.333,348.444,1.000,1388321,0,0,1,625570213,1\n
1,1388321,3,14013.667,2888.000,348.333,1.000,1388322,0,0,1,625570213,1\n
2,1388322,3,14014.444,2888.667,348.889,1.000,1388323,0,0,1,625570213,1\n
3,1388323,3,14015.667,2889.000,348.667,1.000,1388324,0,0,1,625570213,1\n
4,1388324,3,14017.111,2889.000,348.111,1.000,1388325,0,0,1,625570213,1\n


In [7]:
ratioxy=goal_xyz/xy
ratioz= goal_xyz/z
# calculating the downsampling factor
# based on 0.8 x 0.8 x 5um imaging resolution (x y and z)
# downsampling to match the 25um voxel allen reference template

message = (f"Sample resolution is {xy, xy, z} um in x y z. "
f"downsampling to {goal_xyz} um. "
f"dowmsample ratio is xy = {ratioxy} and z = {ratioz}.")

message

'Sample resolution is (0.8, 0.8, 5.0) um in x y z. downsampling to 25.0 um. dowmsample ratio is xy = 31.25 and z = 5.0.'

In [9]:
ds_x= pd.to_numeric(annotation_df['x'])
ds_x=ds_x/ratioxy
ds_xround=ds_x.astype(int).astype(str)
ds_y= pd.to_numeric(annotation_df['y'])
ds_y=ds_y/ratioxy
ds_yround=ds_y.astype(int).astype(str)

ds_z= pd.to_numeric(annotation_df['z'])
ds_z=ds_z/ratioz
ds_zround=ds_z.astype(int).astype(str)

# rounds by astype int, then conver to string

In [10]:
ds_coordinates= pd.DataFrame(columns=['x','y','z'])
ds_coordinates['x']=ds_xround
ds_coordinates['y']=ds_yround
ds_coordinates['z']=ds_zround
# put in to panda dataframe just in case for future use

In [11]:
q = [' '.join(x) for x in zip(ds_xround,ds_yround,ds_zround)]
# create a list where row1= x1 y1 z1 row2= x2 y2 z2 and etc 
# for writing to the text file

In [12]:
num_row=len(annotation_df.index)

out_name= outdir[3:]+ f'_{goal_xyz}voxel_trace.txt'
out_name

'AL080_25.0voxel_trace.txt'

In [13]:
f=open(outdir+'/'+out_name,'w+')
f.write('point'+'\n')
f.write(str(num_row)+'\n')

for lines in q:
    f.write(lines+'\n')

f.close()

#works, yay

In [133]:
#attempt to find endings for each branch
# the #n of the ending should not be a parent for something else

#first put these 2 columns in to list
list_n= annotation_df['#n'].tolist()
list_parent= annotation_df['parent'].tolist()

ending_list=[]
# compare each value in the #n column to the parent list, if it is not present then it should be an ending
for N,values in enumerate (list_n):
    if values not in list_parent:
        ending_list.append(N)
print(f'there are {len(ending_list)} endings')
        

there are 1219 endings
